!pip install ultralytics

!pip install fiftyone

In [1]:
import fiftyone as fo
import fiftyone.zoo as foz
from fiftyone import ViewField as F

C:\Users\knuyh\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


## roboflow로 custom dataset(sweatshirt), openimagev7로 21개 dataset 다운

- dataset.yaml ; train의 경로를 .. -> . 로 변경

### coupang + youtube  roboflow merge => custom dataset (sweatshirt)

In [39]:
name = 'cust'
dataset_dir = r'D:\프로젝트\데이터\YOLO_coupang' # dataset이 있는 directory 경로
splits = ['train', 'val', 'test']


dataset = fo.Dataset(name)
for split in splits :
    dataset.add_dir(
        dataset_dir = dataset_dir,
        dataset_type = fo.types.YOLOv5Dataset, #  YOLOv8 = YOLOv5
        split = split,
        tags = split)

 100% |█████████████████| 758/758 [1.4s elapsed, 0s remaining, 544.1 samples/s]         
 100% |█████████████████| 324/324 [627.9ms elapsed, 0s remaining, 518.4 samples/s]      
 100% |█████████████████| 145/145 [294.8ms elapsed, 0s remaining, 491.8 samples/s]      


In [40]:
dataset_dir2 = r'D:\프로젝트\데이터\YOLO_youtube'
# 다른 class가 있으면 원래 dataset으로 불러들인 것에 덧붙이는 형식이다.
for split in splits :
    dataset.merge_dir(
            dataset_dir = dataset_dir2,
            dataset_type = fo.types.YOLOv5Dataset,
            split = split,
            tags = split)

# 현재 dataset 객체로 있는 것을 export해서 저장을 해야 합니다.
for split in splits :
    dataset.export(
            export_dir = r'D:\프로젝트\데이터\custom',
            split = split,
            dataset_type = fo.types.YOLOv5Dataset)

 100% |███████████████| 1109/1109 [4.9s elapsed, 0s remaining, 199.8 samples/s]      
 100% |█████████████████| 324/324 [715.9ms elapsed, 0s remaining, 452.6 samples/s]      
 100% |█████████████████| 164/164 [403.2ms elapsed, 0s remaining, 406.8 samples/s]      
 100% |███████████████| 2824/2824 [1.3m elapsed, 0s remaining, 17.0 samples/s]       
Directory 'D:\프로젝트\데이터\custom' already exists; export will be merged with existing files
 100% |███████████████| 2824/2824 [1.9m elapsed, 0s remaining, 26.3 samples/s]       
Directory 'D:\프로젝트\데이터\custom' already exists; export will be merged with existing files
 100% |███████████████| 2824/2824 [1.6m elapsed, 0s remaining, 19.9 samples/s]      


### openimagev7 (21개 cat) + custom dataset (1개 cat / sweatshirts roboflow)
- 21개 = 20개는 coco와 openimagev7 겹치는 것 + 1개는 겹치지 않는 것 중 openimagev7에 있는 것

* 20GB 파일 받고, fiftyone에 openimagev7 부름  

import os  
classes = ['Bowl', 'Backpack', 'Laptop', 'Oven', 'Toaster', 'Mouse', 'Bottle',  
         'Tennis racket', 'Clock', 'Tie', 'Wine glass', 'Spoon', 'Scissors', 'Toothbrush',  
         'Refrigerator', 'Couch', 'Chair', 'Umbrella', 'Knife', 'Suitcase','Sunglasses']  
  
splits = ['train', 'validation', 'test']  
  
* 데이터셋을 저장할 디렉토리 정의  

dataset_dir = r'D:\프로젝트\데이터'  
  
* FiftyOne에서 데이터셋을 저장할 디렉토리 설정 

fo.config.dataset_zoo_dir = dataset_dir  
  
* 디렉토리가 없으면 생성 

os.makedirs(dataset_dir, exist_ok=True)  
  
* 데이터를 축적할 FiftyOne 데이터셋 생성

accumulated_dataset = fo.Dataset()  
  
* OpenImages V7 데이터셋에서 특정 클래스의 학습 데이터셋 다운로드

##### 이중for문 -> 오래걸려 kernel dead
for c in classes :

    print(c)  
      
    dataset_name = f"open-images-v7-{c.lower().replace(' ', '-')}-train"  
      
    for split in splits :  
        if split == 'train' :  
            dataset = foz.load_zoo_dataset(    
                "open-images-v7",    
                split=split,    
                classes=c,  
                max_samples = 5000,  
                label_types=["detections"],   
                shuffle = True,  
                only_matching=True,  
                dataset_name = dataset_name)    
            
        elif split == 'validation' :  
            dataset = foz.load_zoo_dataset(    
                "open-images-v7",    
                split=split,    
                classes=c,  
                max_samples = 922,  
                label_types=["detections"],    
                shuffle = True,  
                only_matching=True,  
                dataset_name = dataset_name)    
          
        elif split == 'test' :  
            dataset = foz.load_zoo_dataset(    
                "open-images-v7",    
                split=split,    
                classes=c,  
                max_samples = 459,  
                label_types=["detections"],    
                shuffle = True,  
                only_matching=True) # 데이터셋 다운로드 경로 지정  
              
    accumulated_dataset.add_samples(dataset)
    
      
* YOLO 데이터셋으로 가공

view = accumulated_dataset.filter_labels('ground_truth', fo.ViewField('label').is_in(classes))  
  
splits = ['train', 'val', 'test']   # YOLO 데이터셋으로 가공하면 yaml에서 자동으로 validation -> val 로 변경
   
for split in splits :  

    view.export(  
        export_dir = 'D:\\프로젝트\\데이터\\open-images-v7',  
        dataset_type = fo.types.YOLOv5Dataset,  
        split = split,  
        classes = classes) # 클래스 순서 seed

In [2]:
# train
#20GB 파일 받고, fiftyone에 openimagev7 부름
import os  
classes = ['Bowl', 'Backpack', 'Laptop', 'Oven', 'Toaster', 'Mouse', 'Bottle',  
         'Tennis racket', 'Clock', 'Tie', 'Wine glass', 'Spoon', 'Scissors', 'Toothbrush',  
         'Refrigerator', 'Couch', 'Chair', 'Umbrella', 'Knife', 'Suitcase','Sunglasses']  
    
# 데이터셋을 저장할 디렉토리 정의
dataset_dir = r'D:\프로젝트\데이터'  
  
# FiftyOne에서 데이터셋을 저장할 디렉토리 설정
fo.config.dataset_zoo_dir = dataset_dir  
  
# 디렉토리가 없으면 생성
os.makedirs(dataset_dir, exist_ok=True)  
  
# 데이터를 축적할 FiftyOne 데이터셋 생성
accumulated_dataset = fo.Dataset()  
  
# OpenImages V7 데이터셋에서 특정 클래스의 학습 데이터셋 다운로드
for c in classes : # class를 한번에 지정하면, 전체 class에서 max_samples 개수를 가져옴 (이 코드는 각 클래스마다 max_samples만큼)  
    print(c)  
      
    dataset_name = f"open-images-v7-{c.lower().replace(' ', '-')}-train"  
      
    dataset = foz.load_zoo_dataset(    
                "open-images-v7",    
                split='train',    
                classes=c,  
                max_samples = 5000,  
                label_types=["detections"],   
                shuffle = True,  
                only_matching=True,  
                dataset_name = dataset_name) 
    
accumulated_dataset.add_samples(dataset)

Bowl
 100% |██████████████████████████████████████████████|    4.8Gb/4.8Gb [54.1s elapsed, 0s remaining, 100.6Mb/s]      
Only found 1464 (<5000) samples matching your requirements
 100% |█████████████████| 1464/1464 [2.3m elapsed, 0s remaining, 8.2 files/s]       
Dataset info written to 'D:\프로젝트\데이터\open-images-v7\info.json'
Loading existing dataset 'open-images-v7-bowl-train'. To reload from disk, either delete the existing dataset or provide a custom `dataset_name` to use
Backpack
Only found 715 (<5000) samples matching your requirements
 100% |███████████████████| 715/715 [1.1m elapsed, 0s remaining, 12.8 files/s]      
Dataset info written to 'D:\프로젝트\데이터\open-images-v7\info.json'
Loading existing dataset 'open-images-v7-backpack-train'. To reload from disk, either delete the existing dataset or provide a custom `dataset_name` to use
Laptop
Found 9 images, downloading the remaining 4991
 100% |█████████████████| 4991/4991 [7.1m elapsed, 0s remaining, 10.0 files/s]      
Dataset i

 100% |█████████████████| 3284/3284 [4.1m elapsed, 0s remaining, 12.2 files/s]      
Dataset info written to 'D:\프로젝트\데이터\open-images-v7\info.json'
Loading existing dataset 'open-images-v7-couch-train'. To reload from disk, either delete the existing dataset or provide a custom `dataset_name` to use
Chair
Found 274 images, downloading the remaining 4726
 100% |█████████████████| 4726/4726 [6.0m elapsed, 0s remaining, 7.0 files/s]       
Dataset info written to 'D:\프로젝트\데이터\open-images-v7\info.json'
Loading existing dataset 'open-images-v7-chair-train'. To reload from disk, either delete the existing dataset or provide a custom `dataset_name` to use
Umbrella
Only found 3357 (<5000) samples matching your requirements
Found 36 images, downloading the remaining 3321
 100% |█████████████████| 3321/3321 [4.2m elapsed, 0s remaining, 16.2 files/s]      
Dataset info written to 'D:\프로젝트\데이터\open-images-v7\info.json'
Loading existing dataset 'open-images-v7-umbrella-train'. To reload from disk, 

['655f1ce2ffdf1b1f878ea83c',
 '655f1ce2ffdf1b1f878ea840',
 '655f1ce2ffdf1b1f878ea841',
 '655f1ce2ffdf1b1f878ea849',
 '655f1ce2ffdf1b1f878ea84a',
 '655f1ce2ffdf1b1f878ea84b',
 '655f1ce2ffdf1b1f878ea84c',
 '655f1ce2ffdf1b1f878ea861',
 '655f1ce2ffdf1b1f878ea862',
 '655f1ce2ffdf1b1f878ea863',
 '655f1ce2ffdf1b1f878ea864',
 '655f1ce2ffdf1b1f878ea865',
 '655f1ce2ffdf1b1f878ea866',
 '655f1ce2ffdf1b1f878ea867',
 '655f1ce2ffdf1b1f878ea868',
 '655f1ce2ffdf1b1f878ea87e',
 '655f1ce2ffdf1b1f878ea87f',
 '655f1ce2ffdf1b1f878ea880',
 '655f1ce2ffdf1b1f878ea881',
 '655f1ce2ffdf1b1f878ea882',
 '655f1ce2ffdf1b1f878ea883',
 '655f1ce2ffdf1b1f878ea884',
 '655f1ce2ffdf1b1f878ea885',
 '655f1ce2ffdf1b1f878ea886',
 '655f1ce2ffdf1b1f878ea887',
 '655f1ce2ffdf1b1f878ea888',
 '655f1ce2ffdf1b1f878ea889',
 '655f1ce2ffdf1b1f878ea88a',
 '655f1ce2ffdf1b1f878ea88b',
 '655f1ce2ffdf1b1f878ea88c',
 '655f1ce2ffdf1b1f878ea88d',
 '655f1ce2ffdf1b1f878ea8bf',
 '655f1ce2ffdf1b1f878ea8c0',
 '655f1ce2ffdf1b1f878ea8c1',
 '655f1ce2ffdf

In [3]:
# YOLO 데이터셋으로 가공
view = accumulated_dataset.filter_labels('ground_truth', fo.ViewField('label').is_in(classes))

view.export(
    export_dir = 'D:\\프로젝트\\데이터\\open-images-v7',
    dataset_type = fo.types.YOLOv5Dataset,
    split = 'train',
    classes = classes) # 클래스 순서 seed

Directory 'D:\프로젝트\데이터\open-images-v7' already exists; export will be merged with existing files
 100% |███████████████| 5000/5000 [5.1m elapsed, 0s remaining, 16.3 samples/s]      


In [6]:
# validation
#20GB 파일 받고, fiftyone에 openimagev7 부름
import os  
classes = ['Bowl', 'Backpack', 'Laptop', 'Oven', 'Toaster', 'Mouse', 'Bottle',  
         'Tennis racket', 'Clock', 'Tie', 'Wine glass', 'Spoon', 'Scissors', 'Toothbrush',  
         'Refrigerator', 'Couch', 'Chair', 'Umbrella', 'Knife', 'Suitcase','Sunglasses']  
    
# 데이터셋을 저장할 디렉토리 정의
dataset_dir = r'D:\프로젝트\데이터'  
  
# FiftyOne에서 데이터셋을 저장할 디렉토리 설정
fo.config.dataset_zoo_dir = dataset_dir  
  
# 디렉토리가 없으면 생성
os.makedirs(dataset_dir, exist_ok=True)  
  
# 데이터를 축적할 FiftyOne 데이터셋 생성
accumulated_dataset = fo.Dataset()  
  
# OpenImages V7 데이터셋에서 특정 클래스의 학습 데이터셋 다운로드
for c in classes : # class를 한번에 지정하면, 전체 class에서 max_samples 개수를 가져옴 (이 코드는 각 클래스마다 max_samples만큼)  
    print(c)  
      
    dataset_name = f"open-images-v7-{c.lower().replace(' ', '-')}-val"  
      
    dataset = foz.load_zoo_dataset(    
                "open-images-v7",    
                split='validation',    
                classes=c,  
                max_samples = 922,  
                label_types=["detections"],   
                shuffle = True,  
                only_matching=True,  
                dataset_name = dataset_name) 
    
accumulated_dataset.add_samples(dataset)

Bowl
Only found 49 (<922) samples matching your requirements
Necessary images already downloaded
Existing download of split 'validation' is sufficient
Loading existing dataset 'open-images-v7-bowl-val'. To reload from disk, either delete the existing dataset or provide a custom `dataset_name` to use
Backpack
Only found 24 (<922) samples matching your requirements
Necessary images already downloaded
Existing download of split 'validation' is sufficient
Loading existing dataset 'open-images-v7-backpack-val'. To reload from disk, either delete the existing dataset or provide a custom `dataset_name` to use
Laptop
Only found 52 (<922) samples matching your requirements
Necessary images already downloaded
Existing download of split 'validation' is sufficient
Loading existing dataset 'open-images-v7-laptop-val'. To reload from disk, either delete the existing dataset or provide a custom `dataset_name` to use
Oven
Only found 35 (<922) samples matching your requirements
Necessary images already

 100% |█████████████████| 134/134 [401.9ms elapsed, 0s remaining, 333.4 samples/s]      


['655f1f51ffdf1b1f878f5a19',
 '655f1f51ffdf1b1f878f5a1c',
 '655f1f51ffdf1b1f878f5a1d',
 '655f1f51ffdf1b1f878f5a24',
 '655f1f51ffdf1b1f878f5a25',
 '655f1f51ffdf1b1f878f5a26',
 '655f1f51ffdf1b1f878f5a27',
 '655f1f52ffdf1b1f878f5a31',
 '655f1f52ffdf1b1f878f5a32',
 '655f1f52ffdf1b1f878f5a33',
 '655f1f52ffdf1b1f878f5a34',
 '655f1f52ffdf1b1f878f5a35',
 '655f1f52ffdf1b1f878f5a36',
 '655f1f52ffdf1b1f878f5a37',
 '655f1f52ffdf1b1f878f5a38',
 '655f1f52ffdf1b1f878f5a4a',
 '655f1f52ffdf1b1f878f5a4b',
 '655f1f52ffdf1b1f878f5a4c',
 '655f1f52ffdf1b1f878f5a4d',
 '655f1f52ffdf1b1f878f5a4e',
 '655f1f52ffdf1b1f878f5a4f',
 '655f1f52ffdf1b1f878f5a50',
 '655f1f52ffdf1b1f878f5a51',
 '655f1f52ffdf1b1f878f5a52',
 '655f1f52ffdf1b1f878f5a53',
 '655f1f52ffdf1b1f878f5a54',
 '655f1f52ffdf1b1f878f5a55',
 '655f1f52ffdf1b1f878f5a56',
 '655f1f52ffdf1b1f878f5a57',
 '655f1f52ffdf1b1f878f5a58',
 '655f1f52ffdf1b1f878f5a59',
 '655f1f52ffdf1b1f878f5a8c',
 '655f1f52ffdf1b1f878f5a8d',
 '655f1f52ffdf1b1f878f5a8e',
 '655f1f52ffdf

In [7]:
# YOLO 데이터셋으로 가공
view = accumulated_dataset.filter_labels('ground_truth', fo.ViewField('label').is_in(classes))

view.export(
    export_dir = 'D:\\프로젝트\\데이터\\open-images-v7',
    dataset_type = fo.types.YOLOv5Dataset,
    split = 'val', # YOLO 형식으로 하면 yaml 파일에서 자동으로 validation에서 val로 바뀜
    classes = classes) # 클래스 순서 seed

Directory 'D:\프로젝트\데이터\open-images-v7' already exists; export will be merged with existing files
 100% |█████████████████| 134/134 [8.6s elapsed, 0s remaining, 18.0 samples/s]      


In [6]:
# test
#20GB 파일 받고, fiftyone에 openimagev7 부름
import os  
classes = ['Bowl', 'Backpack', 'Laptop', 'Oven', 'Toaster', 'Mouse', 'Bottle',  
         'Tennis racket', 'Clock', 'Tie', 'Wine glass', 'Spoon', 'Scissors', 'Toothbrush',  
         'Refrigerator', 'Couch', 'Chair', 'Umbrella', 'Knife', 'Suitcase','Sunglasses']  
    
# 데이터셋을 저장할 디렉토리 정의
dataset_dir = r'D:\프로젝트\데이터'  
  
# FiftyOne에서 데이터셋을 저장할 디렉토리 설정
fo.config.dataset_zoo_dir = dataset_dir  
  
# 디렉토리가 없으면 생성
os.makedirs(dataset_dir, exist_ok=True)  
  
# 데이터를 축적할 FiftyOne 데이터셋 생성
accumulated_dataset = fo.Dataset()  
  
# OpenImages V7 데이터셋에서 특정 클래스의 학습 데이터셋 다운로드
for c in classes : # class를 한번에 지정하면, 전체 class에서 max_samples 개수를 가져옴 (이 코드는 각 클래스마다 max_samples만큼)  
    print(c)  
      
    dataset_name = f"open-images-v7-{c.lower().replace(' ', '-')}-test"  
      
    dataset = foz.load_zoo_dataset(    
                "open-images-v7",    
                split='test',    
                classes=c,  
                max_samples = 459,  
                label_types=["detections"],   
                shuffle = True,  
                only_matching=True,  
                dataset_name = dataset_name) 
    
accumulated_dataset.add_samples(dataset)

Bowl
Only found 147 (<459) samples matching your requirements
 100% |███████████████████| 147/147 [12.3s elapsed, 0s remaining, 11.9 files/s]      
Dataset info written to 'D:\프로젝트\데이터\open-images-v7\info.json'
Loading 'open-images-v7' split 'test'
 100% |█████████████████| 147/147 [340.6ms elapsed, 0s remaining, 431.6 samples/s]      
Dataset 'open-images-v7-bowl-test' created
Backpack
Only found 74 (<459) samples matching your requirements
 100% |█████████████████████| 74/74 [8.5s elapsed, 0s remaining, 11.9 files/s]      
Dataset info written to 'D:\프로젝트\데이터\open-images-v7\info.json'
Loading 'open-images-v7' split 'test'
 100% |███████████████████| 74/74 [157.8ms elapsed, 0s remaining, 468.9 samples/s]     
Dataset 'open-images-v7-backpack-test' created
Laptop
Only found 197 (<459) samples matching your requirements
 100% |███████████████████| 197/197 [17.8s elapsed, 0s remaining, 13.5 files/s]      
Dataset info written to 'D:\프로젝트\데이터\open-images-v7\info.json'
Loading 'open-images

 100% |███████████████████| 124/124 [11.5s elapsed, 0s remaining, 10.6 files/s]      
Dataset info written to 'D:\프로젝트\데이터\open-images-v7\info.json'
Loading 'open-images-v7' split 'test'
 100% |█████████████████| 125/125 [218.2ms elapsed, 0s remaining, 573.0 samples/s]      
Dataset 'open-images-v7-couch-test' created
Chair
Found 35 images, downloading the remaining 424
 100% |███████████████████| 424/424 [42.7s elapsed, 0s remaining, 5.1 files/s]       
Dataset info written to 'D:\프로젝트\데이터\open-images-v7\info.json'
Loading 'open-images-v7' split 'test'
 100% |█████████████████| 459/459 [1.3s elapsed, 0s remaining, 347.8 samples/s]         
Dataset 'open-images-v7-chair-test' created
Umbrella
Only found 106 (<459) samples matching your requirements
Found 7 images, downloading the remaining 99
 100% |█████████████████████| 99/99 [11.2s elapsed, 0s remaining, 8.6 files/s]      
Dataset info written to 'D:\프로젝트\데이터\open-images-v7\info.json'
Loading 'open-images-v7' split 'test'
 100% |███

['655e41b12d1b502e7ce5ffe0',
 '655e41b12d1b502e7ce5ffe2',
 '655e41b12d1b502e7ce5ffe5',
 '655e41b12d1b502e7ce5ffe6',
 '655e41b12d1b502e7ce5ffed',
 '655e41b12d1b502e7ce5ffee',
 '655e41b12d1b502e7ce5ffef',
 '655e41b12d1b502e7ce5fff0',
 '655e41b12d1b502e7ce60000',
 '655e41b12d1b502e7ce60001',
 '655e41b12d1b502e7ce60002',
 '655e41b12d1b502e7ce60003',
 '655e41b12d1b502e7ce60004',
 '655e41b12d1b502e7ce60005',
 '655e41b12d1b502e7ce60006',
 '655e41b12d1b502e7ce60007',
 '655e41b12d1b502e7ce6001c',
 '655e41b12d1b502e7ce6001d',
 '655e41b12d1b502e7ce6001e',
 '655e41b12d1b502e7ce6001f',
 '655e41b12d1b502e7ce60020',
 '655e41b12d1b502e7ce60021',
 '655e41b12d1b502e7ce60022',
 '655e41b12d1b502e7ce60023',
 '655e41b12d1b502e7ce60024',
 '655e41b12d1b502e7ce60025',
 '655e41b12d1b502e7ce60026',
 '655e41b12d1b502e7ce60027',
 '655e41b12d1b502e7ce60028',
 '655e41b12d1b502e7ce60029',
 '655e41b12d1b502e7ce6002a',
 '655e41b12d1b502e7ce6002b',
 '655e41b12d1b502e7ce6005e',
 '655e41b12d1b502e7ce6005f',
 '655e41b12d1b

In [7]:
# YOLO 데이터셋으로 가공
view = accumulated_dataset.filter_labels('ground_truth', fo.ViewField('label').is_in(classes))

view.export(
    export_dir = 'D:\\프로젝트\\데이터\\open-images-v7',
    dataset_type = fo.types.YOLOv5Dataset,
    split = 'test',
    classes = classes) # 클래스 순서 seed

Directory 'D:\프로젝트\데이터\open-images-v7' already exists; export will be merged with existing files
 100% |█████████████████| 439/439 [10.0s elapsed, 0s remaining, 47.1 samples/s]      


In [9]:
name = 'final_datasets'
dataset_dir3 = r'D:\프로젝트\데이터\custom'
splits = ['train', 'val', 'test']

dataset = fo.Dataset(name)
for split in splits :
    dataset.add_dir(
        dataset_dir = dataset_dir3,
        dataset_type = fo.types.YOLOv5Dataset,
        split = split,
        tags = split)

 100% |███████████████| 2824/2824 [5.6s elapsed, 0s remaining, 503.0 samples/s]      
 100% |███████████████| 2824/2824 [6.0s elapsed, 0s remaining, 456.3 samples/s]      
 100% |███████████████| 2824/2824 [5.7s elapsed, 0s remaining, 473.5 samples/s]      


In [10]:
dataset_dir4 = r'D:\프로젝트\데이터\open-images-v7'
splits = ['train', 'val', 'test']

for split in splits :
    dataset.merge_dir(
            dataset_dir = dataset_dir4,
            dataset_type = fo.types.YOLOv5Dataset,
            split = split,
            tags = split)

    
nclasses = ['Bowl', 'Backpack', 'Laptop', 'Oven', 'Toaster', 'Mouse', 'Bottle',  
         'Tennis racket', 'Clock', 'Tie', 'Wine glass', 'Spoon', 'Scissors', 'Toothbrush',  
         'Refrigerator', 'Couch', 'Chair', 'Umbrella', 'Knife', 'Suitcase','Sunglasses', 'sweatshirts']  
    
for split in splits :
    nview = dataset.match(fo.ViewField('tags').contains(split))
    nview.export(
            export_dir = r'C:\Users\knuyh\Desktop\민지\고려대학교 교육\프로젝트\personal\MinjiKim\feasibility_test\Data',
            split = split,
            dataset_type = fo.types.YOLOv5Dataset,
            classes = nclasses)

 100% |███████████████| 5000/5000 [13.0s elapsed, 0s remaining, 339.6 samples/s]      
 100% |█████████████████| 134/134 [377.9ms elapsed, 0s remaining, 354.6 samples/s]      
 100% |█████████████████| 439/439 [1.3s elapsed, 0s remaining, 347.6 samples/s]         
Directory 'C:\Users\knuyh\Desktop\민지\고려대학교 교육\프로젝트\personal\MinjiKim\feasibility_test\Data' already exists; export will be merged with existing files
 100% |███████████████| 7824/7824 [4.0m elapsed, 0s remaining, 35.4 samples/s]      
Directory 'C:\Users\knuyh\Desktop\민지\고려대학교 교육\프로젝트\personal\MinjiKim\feasibility_test\Data' already exists; export will be merged with existing files
 100% |███████████████| 2958/2958 [1.3m elapsed, 0s remaining, 38.1 samples/s]       
Directory 'C:\Users\knuyh\Desktop\민지\고려대학교 교육\프로젝트\personal\MinjiKim\feasibility_test\Data' already exists; export will be merged with existing files
 100% |███████████████| 3263/3263 [1.0m elapsed, 0s remaining, 24.6 samples/s]       
